# Lendo atos e fazendo o pré-processamento

Leitura dos atos que foram encontrados em algum DODF. 

Estes atos foram retirados da versão 3 da base de atos de pessoal: https://github.com/UnB-KnEDLe/datasets/blob/master/anotacoes_atos_de_pessoal.md

In [1]:
import pandas as pd

df = pd.read_csv('found.csv').filter(['tipo_rel', 'texto_rel', 'found'], axis = 1)

df

,tipo_rel,texto_rel,found
0,Ato_Substituicao,"DESIGNAR ROBERTO MARTINS DE MELO, Inspetor Tec...",DODF 129 07-07-2015 2.txt
1,Ato_Substituicao,"DESIGNAR LUCINEIDE SILVA DE SOUZA MAGALHAES, T...",DODF 129 07-07-2015 2.txt
2,Ato_Substituicao,"DESIGNAR ESTEVAO CAPUTO E OLIVEIRA, Auditor-Fi...",DODF 129 07-07-2015 2.txt
3,Ato_Substituicao,"DESIGNAR FERNANDO CARVALHO ANTERO, Auditor-Fis...",DODF 129 07-07-2015 2.txt
4,Ato_Substituicao,"RESOLVE: DESIGNAR ANDERSON MENDES BORGES, Audi...",DODF 129 07-07-2015 2.txt
...,...,...,...
9042,Ato_Abono_Permanencia,CONCEDER abono de permanencia \nequivalente ao...,DODF 233 07-11-2014 2.txt
9043,Ato_Retificacao_Efetivo,"Na Portaria no 387, de 31 de outubro de 2014, ...",DODF 233 07-11-2014 2.txt
9044,Ato_Substituicao,"DESIGNAR FABIOLA DE MORAES TRAVASSOS, matri-\n...",DODF 233 07-11-2014 2.txt
9045,Ato_Substituicao,"DESIGNAR DORACINA APARECIDA DA SILVA, matricul...",DODF 233 07-11-2014 2.txt


In [2]:
df['texto_rel'] = df['texto_rel'].apply(lambda x: x.replace('\n', ' ').strip())

In [3]:
tipos_de_ato = df['tipo_rel'].unique()
tipos_de_ato

array(['Ato_Substituicao', 'Ato_Abono_Permanencia',
       'Ato_Retificacao_Efetivo', 'Ato_Cessao', 'Ato_Reversao',
       'Ato_Exoneracao_Comissionado', 'Ato_Tornado_Sem_Efeito_Exo_Nom',
       'Ato_Nomeacao_Comissionado', 'Ato_Retificacao_Comissionado',
       'Ato_Exoneracao_Efetivo', 'Ato_Tornado_Sem_Efeito_Apo',
       'Ato_Nomeacao_Efetivo'], dtype=object)

In [4]:
letras = [chr(c) for c in range(ord('a'), ord('z') + 1)]
numeros = [chr(c) for c in range(ord('0'), ord('9') + 1)]
simbolos = ['(', ',', '.', '/', '-'] # simbolos que começam palavras

alfanumerico = letras + numeros
todos = letras + numeros + simbolos + [' ']

print(letras)
print(numeros)
print(simbolos)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['(', ',', '.', '/', '-']


# Lendo DODFs e fazendo o pré-processamento

Leitura dos DODFs em que algum ato foi encontrado.

In [5]:
import os

files = os.listdir('found')

dodfs = {}
for file in files:
    with open(f'found/{file}', 'r') as f:
        conteudo = f.read()
    dodfs[file] = conteudo.replace('\n', ' ').strip()

In [6]:
dodfs = pd.DataFrame([(k, dodfs[k]) for k in dodfs.keys()], columns = ['nome', 'conteudo'])
dodfs

,nome,conteudo
0,DODF 187 14-09-2012 2.txt,Pagina 17 Diario Oficial do Distrito Federal...
1,DODF 139 08-07-2013 3.txt,Pagina 66 Diario Oficial do Distrito Federal...
2,DODF 173 08-09-2015 2.txt,PAGINA 28 Diario Oficial do Distrito Federal...
3,DODF 204 29-09-2014 2.txt,PAGINA 22 Diario Oficial do Distrito Federal...
4,DODF 010 13-01-2015 2.txt,PAGINA 14 Diario Oficial do Distrito Federal...
...,...,...
123,DODF 244 21-11-2013 2.txt,Pagina 37 Diario Oficial do Distrito Federal...
124,DODF 142 19-07-2012 2.txt,PAGINA 38 Diario Oficial do Distrito Federal...
125,DODF 094 15-05-2012 2.txt,PAGINA 26 Diario Oficial do Distrito Federal...
126,DODF 233 07-11-2014 2.txt,PAGINA 21 Diario Oficial do Distrito Federal...


# Definindo os limites das palavras

Função para definir os limites de cada palavra em uma sentença.

Não foi utilizado o tokenizador do nltk porque foi observado que ele não separava bem em alguns casos.

In [7]:
def limites(sentence):
    lim = []
    if sentence[0] != ' ':
        lim.append(0)
        
    for i in range(1, len(sentence)):
        atual = sentence[i].lower()
        anterior = sentence[i-1].lower()
        
        if atual in letras and anterior not in letras:
            lim.append(i)
        elif atual in numeros and anterior not in numeros:
            lim.append(i)
        elif atual in simbolos:
            lim.append(i)
        elif atual not in todos and anterior in letras:
            lim.append(i)
    return lim

In [8]:
def split_sentence(sentence):
    lim = limites(sentence)
    lim.append(len(sentence))
    
    words = []
    for i in range(1, len(lim)):
        words.append(sentence[lim[i-1]:lim[i]].strip())
    return words

In [9]:
texto = df['texto_rel'][0].replace('\n', ' ')
texto

'DESIGNAR ROBERTO MARTINS DE MELO, Inspetor Tecnico de Controle Interno, matricula  25.856-3, para substituir IVANILDA SOUSA PEREIRA DE MESQUITA, Inspetor Tecnico de  Controle Interno, matricula 25.810-5, Coordenador, Simbolo CNE-06, da Coordenacao de Orien- tacao, Controle e Analise Contabil da Administracao Direta, da Subsecretaria de Contabilidade,  da Secretaria de Estado de Fazenda do Distrito Federal, no periodo de 02 a 31 de julho de 2015,  por motivo de ferias regulamentares.'

In [10]:
split_sentence(texto)

['DESIGNAR',
 'ROBERTO',
 'MARTINS',
 'DE',
 'MELO',
 ',',
 'Inspetor',
 'Tecnico',
 'de',
 'Controle',
 'Interno',
 ',',
 'matricula',
 '25',
 '.',
 '856',
 '-',
 '3',
 ',',
 'para',
 'substituir',
 'IVANILDA',
 'SOUSA',
 'PEREIRA',
 'DE',
 'MESQUITA',
 ',',
 'Inspetor',
 'Tecnico',
 'de',
 'Controle',
 'Interno',
 ',',
 'matricula',
 '25',
 '.',
 '810',
 '-',
 '5',
 ',',
 'Coordenador',
 ',',
 'Simbolo',
 'CNE',
 '-',
 '06',
 ',',
 'da',
 'Coordenacao',
 'de',
 'Orien',
 '-',
 'tacao',
 ',',
 'Controle',
 'e',
 'Analise',
 'Contabil',
 'da',
 'Administracao',
 'Direta',
 ',',
 'da',
 'Subsecretaria',
 'de',
 'Contabilidade',
 ',',
 'da',
 'Secretaria',
 'de',
 'Estado',
 'de',
 'Fazenda',
 'do',
 'Distrito',
 'Federal',
 ',',
 'no',
 'periodo',
 'de',
 '02',
 'a',
 '31',
 'de',
 'julho',
 'de',
 '2015',
 ',',
 'por',
 'motivo',
 'de',
 'ferias',
 'regulamentares',
 '.']

# Obtendo labels

Aqui geraremos as labels de cada palavra nos DODFs, de acordo com os atos lidos.

In [11]:
def binary_search(v, a): # Retorna o índice do maior elemento em v menor que a
    l = 0
    r = len(v) - 1
    while l <= r:
        m = (l+r)//2
        if v[m] == a:
            return m
        elif v[m] > a:
            r = m - 1
        else:
            l = m + 1
    return -1

In [12]:
not_found = 0
found = 0

labels_list = []
for file_index in dodfs.index:
    file_name = dodfs['nome'][file_index]
    file_content = dodfs['conteudo'][file_index]
    
    atos = df.loc[df['found'] == file_name]
    word_pos = limites(file_content)
    labels = ['O' for _ in word_pos]
    
    for i in atos.index:
        entidade = str(atos['texto_rel'][i])
        tipo = atos['tipo_rel'][i]
        
        pos_find = file_content.find(entidade)
        pos = binary_search(word_pos, pos_find)
        
        if pos == -1:
            pos = binary_search(word_pos, pos_find - 1)
            if pos == -1:
                raise Exception('Ato não encontrado')
            
        len_entidades = len(limites(entidade))

        labels[pos] = 'B-%s' %tipo
        for j in range(pos + 1, pos + len_entidades - 1):
            labels[j] = 'I-%s' %tipo
        labels[pos + len_entidades - 1] = 'E-%s' %tipo
        
    labels_list.append(' '.join(labels))
    
dodfs['labels'] = labels_list

In [13]:
dodfs['labels']

0      O O O O O O O O O O O O O O O O O O O O O O O ...
1      O O O O O O O O O O O O O O O O O O O O O O O ...
2      O O O O O O O O O O O O O O O O O O O O O O O ...
3      O O O O O O O O O O O O O O O O O O O O O O O ...
4      O O O O O O O O O O O O O O O O O O O O O O O ...
                             ...                        
123    O O O O O O O O O O O O O O O O O O O O O O O ...
124    O O O O O O O O O O O O O O O O O O O O O O O ...
125    O O O O O O O O O O O O O O O O O O O O O O O ...
126    O O O O O O O O O O O O O O O O O O O O O O O ...
127    O O O O O O O O O O O O O O O O O O O O O O O ...
Name: labels, Length: 128, dtype: object

In [14]:
texto = split_sentence(dodfs.loc[0, 'conteudo'])
labels = dodfs.loc[0, 'labels'].split()

for i in range(500):
    print(f'{labels[i]} | {texto[i]}')

O | Pagina
O | 17
O | Diario
O | Oficial
O | do
O | Distrito
O | Federal
O | no
O | 187
O | sexta
O | -
O | feira
O | ,
O | 14
O | de
O | setembro
O | de
O | 2012
O | SECAO
O | II
O | DECRETO
O | DE
O | 13
O | DE
O | SETEMBRO
O | DE
O | 2012
O | .
O | O
O | GOVERNADOR
O | DO
O | DISTRITO
O | FEDERAL
O | ,
O | no
O | uso
O | das
O | atribuicoes
O | que
O | lhe
O | confere
O | o
O | artigo
O | 100
O | ,
O | incisos
O | III
O | ,
O | XXVI
O | e
O | XXVII
O | ,
O | da
O | Lei
O | Organica
O | do
O | Distrito
O | Federal
O | ,
O | resolve
O | :
B-Ato_Nomeacao_Comissionado | NOMEAR
I-Ato_Nomeacao_Comissionado | ARQUICELSO
I-Ato_Nomeacao_Comissionado | BITES
I-Ato_Nomeacao_Comissionado | LEAO
I-Ato_Nomeacao_Comissionado | LEITE
I-Ato_Nomeacao_Comissionado | ,
I-Ato_Nomeacao_Comissionado | ocupante
I-Ato_Nomeacao_Comissionado | do
I-Ato_Nomeacao_Comissionado | Cargo
I-Ato_Nomeacao_Comissionado | de
I-Ato_Nomeacao_Comissionado | Natureza
I-Ato_Nomeacao_Comissionado | Espe
I-Ato_Nomeacao_Comissi

# Gerando features

Geração de features para o CRF.

As funções na célula a seguir são funções suporte para reduzir o código.

In [15]:
def number_of_digits(s):
    return sum(c.isdigit() for c in s)

def get_base_feat(word):
    d = {
        'word': word.lower(),
        'is_title': word.istitle(),
        'is_upper': word.isupper(),
        'num_digits': str(number_of_digits(word)),
    }
    return d

def add_base_feat(features, sentence, index, prefix):
    if index >= 0 and index < len(sentence):
        word_feat = get_base_feat(sentence[index])
        for feat in word_feat.keys():
            features[prefix + feat] = word_feat[feat]

In [16]:
def get_features(sentence):
        sent_features = []
        
        for i in range(len(sentence)):
            word = sentence[i]

            word_feat = {
                'bias': 1.0,
                'text_position': i/len(sentence),
            }
            
            add_base_feat(word_feat, sentence, i-4, '-4:')
            add_base_feat(word_feat, sentence, i-3, '-3:')
            add_base_feat(word_feat, sentence, i-2, '-2:')
            add_base_feat(word_feat, sentence, i-1, '-1:')
            
            add_base_feat(word_feat, sentence, i, '')
            
            add_base_feat(word_feat, sentence, i+1, '+1:')
            add_base_feat(word_feat, sentence, i+2, '+2:')
            add_base_feat(word_feat, sentence, i+3, '+3:')
            add_base_feat(word_feat, sentence, i+4, '+4:')
                
            sent_features.append(word_feat)
        
        return sent_features

In [17]:
features_list = []
for i in dodfs.index:
    features = get_features(split_sentence(dodfs['conteudo'][i]))
    features_list.append(features)

dodfs['features'] = features_list

In [18]:
dodfs['features']

0      [{'bias': 1.0, 'text_position': 0.0, 'word': '...
1      [{'bias': 1.0, 'text_position': 0.0, 'word': '...
2      [{'bias': 1.0, 'text_position': 0.0, 'word': '...
3      [{'bias': 1.0, 'text_position': 0.0, 'word': '...
4      [{'bias': 1.0, 'text_position': 0.0, 'word': '...
                             ...                        
123    [{'bias': 1.0, 'text_position': 0.0, 'word': '...
124    [{'bias': 1.0, 'text_position': 0.0, 'word': '...
125    [{'bias': 1.0, 'text_position': 0.0, 'word': '...
126    [{'bias': 1.0, 'text_position': 0.0, 'word': '...
127    [{'bias': 1.0, 'text_position': 0.0, 'word': '...
Name: features, Length: 128, dtype: object

# Funções de validação do modelo

Faremos a validação por kfold cross-validation, utilizando 5 folds. Para cada fold, utilizaremos o random search para encontrar os melhores parâmetros. Ao final, os resultados são ordenados de acordo com o score obtido, e o resultado do meio (nesse caso, o 3º) é retornado.

In [19]:
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import KFold

In [20]:
def RandomizedSearchCRF(X, y, labels, n_iter = 3):
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        max_iterations=150,
        all_possible_transitions=True
    )
    
    params_space = {
        'c1': scipy.stats.expon(scale=0.5),
        'c2': scipy.stats.expon(scale=0.05),
        'epsilon': scipy.stats.expon(scale=1e-4),
    }
    
    f1_scorer = make_scorer(metrics.flat_f1_score,
                            average='weighted', labels=labels)

    clf = RandomizedSearchCV(crf, params_space, cv = 5, n_jobs = -1, n_iter = n_iter, scoring = f1_scorer, verbose = 1)
    clf.fit(X, y)
    
    return clf.best_estimator_

In [21]:
def KFoldCrossValidation(X, y, n_splits = 5, n_iter = 3):
    kf = KFold(n_splits = n_splits, random_state = 87, shuffle = True)
    
    results = []
    
    labels = list(np.unique(np.concatenate([np.array(i) for i in y])))
    labels.remove('O')
    
    for train, test in kf.split(X, y):
        X_train, y_train = X[train], y[train]
        X_test, y_test = X[test], y[test]
        
        clf = RandomizedSearchCRF(X_train, y_train, labels, n_iter = n_iter)
        
        y_true, y_pred = y_test, clf.predict(X_test)
        score = metrics.flat_f1_score(y_true, y_pred, average='weighted', labels=labels)
        report = metrics.flat_classification_report(y_true, y_pred, labels = labels, digits = 3)
        
        results.append({'clf': clf, 'score': score, 'report': report})
        
    results.sort(key = lambda x: x['score'])
    
    middle = ( n_splits + 1 ) // 2

    return results[middle]

# Geração dos modelos

As funções save e check a seguir são funções suporte.

Para a geração de um modelo do tipo de ato X, todas as labels diferentes de B-X, I-X e E-X são convertidas em O.  
Dessa forma, esse modelo aprenderá a identificar apenas o ato X.

In [22]:
import pickle
def save(model, name):
    with open(name + '.pkl', 'wb') as file:
        pickle.dump(model, file)
        
def check(name):
    try:
        file = open(name + '.pkl', 'rb')
        file.close()
        return True
    except FileNotFoundError:
        return False

In [23]:
for ato in tipos_de_ato:
    if check(ato): # Pula esse ato se ele já tiver sido criado
        continue
    save(None, ato) # Se estivermos gerando vários modelos em paralelo, isso faz com que outros notebooks pulem esse ato.
    
    selected = [f'B-{ato}', f'I-{ato}', f'E-{ato}']

    counts = []
    for i in dodfs['labels']:
        counts.append(i.count(f'B-{ato}'))
                      
    dodfs['count'] = counts
                      
    t = dodfs.sort_values('count', ascending = False).head(50).reset_index(drop = True)
    
    X = t['features']
    y = t['labels'].apply(lambda x: x.split())
    
    for i in y.index:
        for j in range(len(y[i])):
            if y[i][j] not in selected:
                y[i][j] = 'O'
            
    report = KFoldCrossValidation(X, y, n_iter = 2)
    save(report, ato)

In [24]:
for ato in tipos_de_ato:
    with open(ato + '.pkl', 'rb') as file:
        model = pickle.load(file)
        print(f'{ato}: {model["score"]:.3f}')

Ato_Substituicao: 0.948
Ato_Abono_Permanencia: 0.905
Ato_Retificacao_Efetivo: 0.802
Ato_Cessao: 0.834
Ato_Reversao: 0.600
Ato_Exoneracao_Comissionado: 0.964
Ato_Tornado_Sem_Efeito_Exo_Nom: 0.956
Ato_Nomeacao_Comissionado: 0.949
Ato_Retificacao_Comissionado: 0.756
Ato_Exoneracao_Efetivo: 0.876
Ato_Tornado_Sem_Efeito_Apo: 0.371
Ato_Nomeacao_Efetivo: 0.345
